In [1]:
import pandas as pd
import numpy as np

In [2]:
demant_df = pd.read_csv('data/bedarfe.csv', sep=';')
demant_df.head()


,Id,Activity,Strecke,From,To,Direction,Length,Depth
0,1,0,0,1720,2274,1,554,800
1,2,0,0,2341,3427,1,1086,800
2,3,0,0,3472,3700,1,228,800
3,4,0,0,3425,3700,2,275,800
4,5,0,0,2338,3380,2,1042,800


In [3]:
raw_container_df = pd.read_csv('data/container.csv', sep=';')
raw_container_df.head()

,Id,Date,Strecke,From,To,Btk
0,0,0,22,136,4621,0
1,1,0,30,657,13100,1
2,2,0,30,657,13100,1
3,3,0,30,657,13100,1
4,4,0,36,-491,51000,2


In [4]:
container_df = pd.DataFrame(columns=raw_container_df.columns)
uniqueValues = []

def isRowUnique(row, valuesList):
    for value in valuesList:
        if row["From"] == value["From"] and row["To"] == value["To"]:
            return False
    return True

# Filter unique containers
for index, row in raw_container_df.iterrows():

    if isRowUnique(row, uniqueValues):
        tempValue = {
            "From": row["From"],
            "To": row["To"]
        }
        uniqueValues.append(tempValue)

        container_df = pd.concat([container_df, pd.DataFrame([row])], ignore_index=True)

container_df.head()



,Id,Date,Strecke,From,To,Btk
0,0,0,22,136,4621,0
1,1,0,30,657,13100,1
2,4,0,36,-491,51000,2
3,5,0,2,15000,202401,3
4,6,0,167,25150,34490,4


In [5]:
print("Number of demands:")
print(demant_df.shape[0])
print("")
print("Number of containers:")
original_number_of_containers = container_df.shape[0]
print(original_number_of_containers)

Number of demands:
5574

Number of containers:
2459


Directions

In [6]:
direction1_df = container_df[container_df["From"] < container_df["To"]]
direction1_df["Direction"] = 1
direction2_df = container_df[container_df["From"] > container_df["To"]]
direction2_df["Direction"] = 2

/tmp/ipykernel_18041/451987248.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  direction1_df["Direction"] = 1
/tmp/ipykernel_18041/451987248.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  direction2_df["Direction"] = 2


In [7]:
print("Number of containers in direction 1:")
print(direction1_df.shape[0])
direction1_df.head()

Number of containers in direction 1:
1930


,Id,Date,Strecke,From,To,Btk,Direction
0,0,0,22,136,4621,0,1
1,1,0,30,657,13100,1,1
2,4,0,36,-491,51000,2,1
3,5,0,2,15000,202401,3,1
4,6,0,167,25150,34490,4,1


In [8]:
print("Number of containers in direction 2:")
print(direction2_df.shape[0])
direction2_df.head()

Number of containers in direction 2:
529


,Id,Date,Strecke,From,To,Btk,Direction
64,66,0,46,95100,24500,54,2
101,103,0,163,197549,181000,81,2
141,145,1,92,44159,0,6,2
199,205,1,244,128400,105,0,2
200,206,1,137,158200,47000,81,2


In [9]:
# Swap From and To for direction 2
print("Number of containers in direction 2:")
direction2_df[['From', 'To']] = direction2_df[['To', 'From']]

print(direction2_df.shape[0])
direction2_df.head()

Number of containers in direction 2:
529


/tmp/ipykernel_18041/4151439604.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  direction2_df[['From', 'To']] = direction2_df[['To', 'From']]


,Id,Date,Strecke,From,To,Btk,Direction
64,66,0,46,24500,95100,54,2
101,103,0,163,181000,197549,81,2
141,145,1,92,0,44159,6,2
199,205,1,244,105,128400,0,2
200,206,1,137,47000,158200,81,2


In [10]:
container_df = pd.concat([direction1_df, direction2_df], ignore_index=True)

In [11]:
assert original_number_of_containers == container_df.shape[0]

In [12]:
# Kann ein Container ein Bedarf zum Teil bedienen
def can_container_serve_demand_patially(container, demand):
    if demand["Direction"] != 0 and demand["Direction"] != container["Direction"]:
        return False

    if container["To"] <= demand["From"] or container["From"] >= demand["To"]:
        return False
    
    return True

In [13]:
demand_container_df = pd.DataFrame(columns=["demand", "container"])

In [14]:
for i_index, demand in demant_df.iterrows():
    if i_index % 100 == 0:
        print(f"Searched: {i_index} / {demant_df.shape[0]} rows")
    for j_index, container in container_df.iterrows():
        if can_container_serve_demand_patially(container, demand):
            tempElem = {
                "demand": demand["Id"],
                "container": container["Id"]
            }
            demand_container_df = pd.concat([demand_container_df, pd.DataFrame([tempElem])], ignore_index=True)

Searched: 0 / 5574 rows
Searched: 100 / 5574 rows
Searched: 200 / 5574 rows
Searched: 300 / 5574 rows


KeyboardInterrupt: 

In [162]:
demand_container_df.head()